
<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Three: Classifier training and performance assessment. </h2>	

In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [11]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.


In [12]:
from ipynb.fs.defs.GradProject_NB2 import feature_frame

In [57]:
data = feature_frame()
y = data['Label']
x = data.drop(columns=['Label', 'Image_Id'])
print("Done preprocessing")

Fetching label '.DS_Store'
Fetching label 'airplanes'
Fetching label 'bear'
Fetching label 'blimp'
	Gray image ('blimp_0022.jpg') was loaded, converting to RGB
Fetching label 'comet'
	Gray image ('comet_0006.jpg') was loaded, converting to RGB
	Gray image ('comet_0011.jpg') was loaded, converting to RGB
	Gray image ('comet_0013.jpg') was loaded, converting to RGB
	Gray image ('comet_0021.jpg') was loaded, converting to RGB
	Gray image ('comet_0036.jpg') was loaded, converting to RGB
	Gray image ('comet_0038.jpg') was loaded, converting to RGB
	Gray image ('comet_0041.jpg') was loaded, converting to RGB
	Gray image ('comet_0049.jpg') was loaded, converting to RGB
	Gray image ('comet_0052.jpg') was loaded, converting to RGB
	Gray image ('comet_0053.jpg') was loaded, converting to RGB
	Gray image ('comet_0057.jpg') was loaded, converting to RGB
	Gray image ('comet_0058.jpg') was loaded, converting to RGB
Fetching label 'crab'
	Gray image ('crab_0045.jpg') was loaded, converting to RGB
Fet

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [65]:
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

Logistic Regression

In [66]:
model = LogisticRegression()
model.fit(x_train_scaled, y_train)
preds = model.predict(x_test_scaled)
acc = accuracy_score(preds, y_test)
print(acc)

0.3687707641196013


K-nearest Neighbors

In [67]:
model = KNeighborsClassifier(10)
model.fit(x_train, y_train)
preds = model.predict(x_test)
acc = accuracy_score(preds, y_test)
print(acc)

0.2724252491694352


Classification Tree

In [68]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
preds = model.predict(x_test)
acc = accuracy_score(preds, y_test)
print(acc)

0.3488372093023256


Random Forest

In [89]:
model = RandomForestClassifier(n_estimators=500)
model.fit(x_train, y_train)
preds = model.predict(x_test)
acc = accuracy_score(preds, y_test)
print(acc)

s = model.feature_importances_
index_importance_sorted = sorted(range(len(s)), key=lambda k: s[k], reverse=True)
top_index = index_importance_sorted[:15]

print("\nMost importance features:")
for index in top_index:
    print(f"Feature name: {x_train.columns[index]}, Importance={s[index]}")

0.4186046511627907

Most importance features:
Feature name: Size, Importance=0.016387738225159387
Feature name: AspectRatio, Importance=0.012417995640841133
Feature name: hist_feat_145, Importance=0.00787149114354435
Feature name: hist_feat_218, Importance=0.006572581681469944
Feature name: hist_feat_438, Importance=0.006134669768131224
Feature name: hist_feat_217, Importance=0.005981805179698633
Feature name: hist_feat_365, Importance=0.005552077537883201
Feature name: hist_feat_65, Importance=0.0054711040886798856
Feature name: hist_feat_511, Importance=0.00542331389925581
Feature name: hist_feat_209, Importance=0.00536940437545721
Feature name: hist_feat_282, Importance=0.005131150972314763
Feature name: hist_feat_144, Importance=0.004995087798905965
Feature name: hist_feat_291, Importance=0.004939820340484287
Feature name: hist_feat_201, Importance=0.00493754861145678
Feature name: hist_feat_138, Importance=0.004935420145545727


Support Vector Machine

In [87]:
model = SVC(kernel='rbf',C=10, gamma=0.01)
model.fit(x_train_scaled, y_train)
preds = model.predict(x_test_scaled)
acc = accuracy_score(preds, y_test)
print(acc)

0.37209302325581395
